In [14]:
import os
from PIL import Image

def stitch_images_bfdh(image_folder, output_path, spacing=0):
    # Load all PNG images from the folder
    images = []
    for filename in os.listdir(image_folder):
        if filename.endswith('.png'):
            img_path = os.path.join(image_folder, filename)
            img = Image.open(img_path)
            images.append(img)

    # Sort images by width in descending order for better packing
    images.sort(key=lambda img: img.width, reverse=True)

    # Initialize variables for row-based packing
    rows = []
    current_row = []
    current_width = 0
    max_height_in_row = 0
    max_canvas_width = 0

    max_width = max(img.width for img in images)  # Maximum width of any image
    max_total_height = 0

    # Group images into rows based on width (bin-packing approach)
    for img in images:
        if current_width + img.width + (len(current_row) * spacing) <= max_width:
            # Add image to the current row
            current_row.append(img)
            current_width += img.width
            max_height_in_row = max(max_height_in_row, img.height)
        else:
            # Move to the next row
            rows.append((current_row, current_width, max_height_in_row))
            max_canvas_width = max(max_canvas_width, current_width + (len(current_row) - 1) * spacing)
            max_total_height += max_height_in_row + spacing

            # Reset for the new row
            current_row = [img]
            current_width = img.width
            max_height_in_row = img.height

    # Add the last row
    if current_row:
        rows.append((current_row, current_width, max_height_in_row))
        max_canvas_width = max(max_canvas_width, current_width + (len(current_row) - 1) * spacing)
        max_total_height += max_height_in_row  # Don't add spacing after the last row

    # Create a new blank canvas large enough to hold all rows
    stitched_image = Image.new('RGBA', (max_canvas_width, max_total_height))

    # Variable to keep track of current y-position (vertical stacking)
    y_offset = 0

    # Stitch images row by row
    for row, row_width, row_height in rows:
        x_offset = 0
        for img in row:
            # Paste each image into its row with spacing
            stitched_image.paste(img, (x_offset, y_offset))
            x_offset += img.width + spacing  # Move to the right for the next image
        y_offset += row_height + spacing  # Move down for the next row with spacing

    # Save the stitched image
    stitched_image.save(output_path)
    print(f"Stitched image saved as {output_path}")

# # Run stitch_images_bfdh for each item in the folders_created list
# for folder in folders_created:
#     image_folder = os.path.expanduser(os.path.join('~/Desktop/GNN/outputpng/', folder))
#     output_path = os.path.expanduser(os.path.join('~/Desktop/GNN/outputpng/', folder, f"{folder}.png"))
#     stitch_images_bfdh(image_folder, output_path, spacing=10)

outputfile = r"C:\Users\Admin\Desktop\GNNoutputpng\OneStepDataset\outputpng.png"
# if outputfile exists then delete it
if os.path.exists(outputfile):
    os.remove(outputfile)  # Remove the file
    print(f"File '{outputfile}' has been removed.")
else:
    print(f"File '{outputfile}' does not exist.")

image_folder = r'C:\Users\Admin\Desktop\GNNoutputpng\OneStepDataset'
stitch_images_bfdh(image_folder, outputfile, spacing=10)  # Example with 10px spacing


File 'C:\Users\Admin\Desktop\GNNoutputpng\OneStepDataset\KingRat.png' has been removed.
Stitched image saved as C:\Users\Admin\Desktop\GNNoutputpng\OneStepDataset\KingRat.png
